In [2]:
import pickle

import cv2
import mediapipe as mp
import numpy as np


In [34]:
with open('best_model.pickle', 'rb') as file:
    loaded_model = pickle.load(file)

In [5]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [6]:
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [8]:
labels_dict={0:'a',
          1:'b',
          2:'c',
          3:'d',
           4:'e',
           5:'f',
           6:'g',
           7:'h',
           8:'i',
           9:'j',
           10:'k',
           11:'l',
           12:'m',
           13:'n',
           14:'o',
           15:'p',
           16:'q',
           17:'r',
           18:'s',
           19:'t',
           20:'u',
           21:'v',
           22:'w',
           23:'x',
           24:'y',
           25:'z',
           26:'0',
           27:'1',
           28:'2',
           29:'3',
           30:'4',
           31:'5',
           32:'6',
           33:'7',
           34:'8',
           35:'9'
          }

In [37]:
cap = cv2.VideoCapture(0)
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        data_aux = np.pad(data_aux, (0, 84 - len(data_aux)), mode='constant')

        prediction = loaded_model.predict([np.asarray(data_aux)])

        predicted_character = prediction[0]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)
        

    
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()